In [1]:
import argparse
import pickle
from pathlib import Path

import jax.numpy as jnp
import matplotlib.pyplot as plt
import wandb
from flax.serialization import from_bytes
from jax import random
from tqdm import tqdm

# from cifar100_resnet20_train import make_stuff
from datasets import load_cifar100
from resnet20 import BLOCKS_PER_GROUP, ResNet
from utils import (ec2_get_instance_type, flatten_params, lerp, timeblock, unflatten_params)
from weight_matching import (apply_permutation, resnet20_permutation_spec, weight_matching)


2023-01-16 17:20:44.419742: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-16 17:20:45.197959: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-16 17:20:45.198057: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-16 17:20:45.198065: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
/srv/share/gstoica3/miniconda3/envs/open-mmlab/lib/python3.7/si

In [2]:
!nvidia-smi

Mon Jan 16 17:20:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A40                 On   | 00000000:81:00.0 Off |                    0 |
|  0%   39C    P0    58W / 300W |      0MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import pickle

In [4]:
model_a = ResNet(blocks_per_group=BLOCKS_PER_GROUP["resnet20"],
                   num_classes=512,
                   width_multiplier=4)
model_b = ResNet(blocks_per_group=BLOCKS_PER_GROUP["resnet20"],
                   num_classes=512,
                   width_multiplier=4)
model_new = ResNet(blocks_per_group=BLOCKS_PER_GROUP["resnet20"],
                   num_classes=512,
                   width_multiplier=4)

In [5]:
!nvidia-smi

Mon Jan 16 17:20:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A40                 On   | 00000000:81:00.0 Off |                    0 |
|  0%   39C    P0    58W / 300W |      0MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
def load_pickle(path):
    return pickle.load(open(path, 'rb'))

In [16]:
a_sd = load_pickle(os.path.join(
    '/srv/share/gstoica3/checkpoints/REPAIR',
    'flax_cifar50_1.pkl'
))

b_sd = load_pickle(os.path.join(
    '/srv/share/gstoica3/checkpoints/REPAIR',
    'flax_cifar50_2.pkl'
))

In [6]:
from collections import defaultdict
from typing import NamedTuple

import jax.numpy as jnp
from jax import random

from utils import rngmix

In [7]:
!nvidia-smi

Mon Jan 16 17:20:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A40                 On   | 00000000:81:00.0 Off |                    0 |
|  0%   39C    P0    58W / 300W |      0MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
class PermutationSpec(NamedTuple):
    perm_to_axes: dict
    axes_to_perm: dict

# def mlp_permutation_spec(num_hidden_layers: int) -> PermutationSpec:
#   """We assume that one permutation cannot appear in two axes of the same weight array."""
#   assert num_hidden_layers >= 1
#   return PermutationSpec(
#       perm_to_axes={
#           f"P_{i}": [(f"Dense_{i}/kernel", 1), (f"Dense_{i}/bias", 0), (f"Dense_{i+1}/kernel", 0)]
#           for i in range(num_hidden_layers)
#       },
#       axes_to_perm={
#           "Dense_0/kernel": (None, "P_0"),
#           **{f"Dense_{i}/kernel": (f"P_{i-1}", f"P_{i}")
#              for i in range(1, num_hidden_layers)},
#           **{f"Dense_{i}/bias": (f"P_{i}", )
#              for i in range(num_hidden_layers)},
#           f"Dense_{num_hidden_layers}/kernel": (f"P_{num_hidden_layers-1}", None),
#           f"Dense_{num_hidden_layers}/bias": (None, ),
#       })

def permutation_spec_from_axes_to_perm(axes_to_perm: dict) -> PermutationSpec:
    perm_to_axes = defaultdict(list)
    for wk, axis_perms in axes_to_perm.items():
        for axis, perm in enumerate(axis_perms):
            if perm is not None:
                perm_to_axes[perm].append((wk, axis))
    return PermutationSpec(perm_to_axes=dict(perm_to_axes), axes_to_perm=axes_to_perm)

def mlp_permutation_spec(num_hidden_layers: int) -> PermutationSpec:
    """We assume that one permutation cannot appear in two axes of the same weight array."""
    assert num_hidden_layers >= 1
    return permutation_spec_from_axes_to_perm({
      "Dense_0/kernel": (None, "P_0"),
      **{f"Dense_{i}/kernel": (f"P_{i-1}", f"P_{i}")
         for i in range(1, num_hidden_layers)},
      **{f"Dense_{i}/bias": (f"P_{i}", )
         for i in range(num_hidden_layers)},
      f"Dense_{num_hidden_layers}/kernel": (f"P_{num_hidden_layers-1}", None),
      f"Dense_{num_hidden_layers}/bias": (None, ),
    })

def resnet20_permutation_spec() -> PermutationSpec:
    conv = lambda name, p_in, p_out: {f"{name}/kernel": (None, None, p_in, p_out)}
    norm = lambda name, p: {
        f"{name}/scale": (p, ), 
        f"{name}/bias": (p, ), 
        f"{name}/mean": (p, ), 
        f"{name}/var": (p, )
    }
    dense = lambda name, p_in, p_out: {f"{name}/kernel": (p_in, p_out), f"{name}/bias": (p_out, )}

    # This is for easy blocks that use a residual connection, without any change in the number of channels.
    easyblock = lambda name, p: {
      **conv(f"{name}/conv1", p, f"P_{name}_inner"),
      **norm(f"{name}/norm1", f"P_{name}_inner"),
      **conv(f"{name}/conv2", f"P_{name}_inner", p),
      **norm(f"{name}/norm2", p)
    }

    # This is for blocks that use a residual connection, but change the number of channels via a Conv.
    shortcutblock = lambda name, p_in, p_out: {
      **conv(f"{name}/conv1", p_in, f"P_{name}_inner"),
      **norm(f"{name}/norm1", f"P_{name}_inner"),
      **conv(f"{name}/conv2", f"P_{name}_inner", p_out),
      **norm(f"{name}/norm2", p_out),
      **conv(f"{name}/shortcut/layers_0", p_in, p_out),
      **norm(f"{name}/shortcut/layers_1", p_out),
    }

    return permutation_spec_from_axes_to_perm({
      **conv("conv1", None, "P_bg0"),
      **norm("norm1", "P_bg0"),
      #
      **easyblock("blockgroups_0/blocks_0", "P_bg0"),
      **easyblock("blockgroups_0/blocks_1", "P_bg0"),
      **easyblock("blockgroups_0/blocks_2", "P_bg0"),
      #
      **shortcutblock("blockgroups_1/blocks_0", "P_bg0", "P_bg1"),
      **easyblock("blockgroups_1/blocks_1", "P_bg1"),
      **easyblock("blockgroups_1/blocks_2", "P_bg1"),
      #
      **shortcutblock("blockgroups_2/blocks_0", "P_bg1", "P_bg2"),
      **easyblock("blockgroups_2/blocks_1", "P_bg2"),
      **easyblock("blockgroups_2/blocks_2", "P_bg2"),
      #
      **dense("dense", "P_bg2", None),
    })

In [9]:
def get_permuted_param(ps: PermutationSpec, perm, k: str, params, except_axis=None):
  """Get parameter `k` from `params`, with the permutations applied."""
    w = params[k]
    for axis, p in enumerate(ps.axes_to_perm[k]):
    # Skip the axis we're trying to permute.
        if axis == except_axis:
            continue

        # None indicates that there is no permutation relevant to that axis.
        if p is not None:
            w = jnp.take(w, perm[p], axis=axis)

    return w

def apply_permutation(ps: PermutationSpec, perm, params):
  """Apply a `perm` to `params`."""
  return {k: get_permuted_param(ps, perm, k, params) for k in params.keys()}

def weight_matching(rng,
                    ps: PermutationSpec,
                    params_a,
                    params_b,
                    max_iter=100,
                    init_perm=None,
                    silent=False):
  """Find a permutation of `params_b` to make them match `params_a`."""
  perm_sizes = {p: params_a[axes[0][0]].shape[axes[0][1]] for p, axes in ps.perm_to_axes.items()}

  perm = {p: jnp.arange(n) for p, n in perm_sizes.items()} if init_perm is None else init_perm
  perm_names = list(perm.keys())

  for iteration in range(max_iter):
    progress = False
    for p_ix in random.permutation(rngmix(rng, iteration), len(perm_names)):
      p = perm_names[p_ix]
      n = perm_sizes[p]
      A = jnp.zeros((n, n))
      for wk, axis in ps.perm_to_axes[p]:
        w_a = params_a[wk]
        w_b = get_permuted_param(ps, perm, wk, params_b, except_axis=axis)
        w_a = jnp.moveaxis(w_a, axis, 0).reshape((n, -1))
        w_b = jnp.moveaxis(w_b, axis, 0).reshape((n, -1))
        A += w_a @ w_b.T

      ri, ci = linear_sum_assignment(A, maximize=True)
      assert (ri == jnp.arange(len(ri))).all()

      oldL = jnp.vdot(A, jnp.eye(n)[perm[p]])
      newL = jnp.vdot(A, jnp.eye(n)[ci, :])
      if not silent: print(f"{iteration}/{p}: {newL - oldL}")
      progress = progress or newL > oldL + 1e-12

      perm[p] = jnp.array(ci)

    if not progress:
      break

  return perm

def test_weight_matching():
  """If we just have a single hidden layer then it should converge after just one step."""
  ps = mlp_permutation_spec(num_hidden_layers=1)
  rng = random.PRNGKey(123)
  num_hidden = 10
  shapes = {
      "Dense_0/kernel": (2, num_hidden),
      "Dense_0/bias": (num_hidden, ),
      "Dense_1/kernel": (num_hidden, 3),
      "Dense_1/bias": (3, )
  }
  params_a = {k: random.normal(rngmix(rng, f"a-{k}"), shape) for k, shape in shapes.items()}
  params_b = {k: random.normal(rngmix(rng, f"b-{k}"), shape) for k, shape in shapes.items()}
  perm = weight_matching(rng, ps, params_a, params_b)
  print(perm)


In [10]:
!nvidia-smi

Mon Jan 16 17:20:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A40                 On   | 00000000:81:00.0 Off |                    0 |
|  0%   37C    P8    22W / 300W |      0MiB / 45634MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
[i for i in flatten_params(a_sd).keys() if i.startswith('params')]

['params/conv1/kernel',
 'params/norm1/scale',
 'params/norm1/bias',
 'params/blockgroups_0/blocks_0/conv1/kernel',
 'params/blockgroups_0/blocks_0/norm1/scale',
 'params/blockgroups_0/blocks_0/norm1/bias',
 'params/blockgroups_0/blocks_0/conv2/kernel',
 'params/blockgroups_0/blocks_0/norm2/scale',
 'params/blockgroups_0/blocks_0/norm2/bias',
 'params/blockgroups_0/blocks_1/conv1/kernel',
 'params/blockgroups_0/blocks_1/norm1/scale',
 'params/blockgroups_0/blocks_1/norm1/bias',
 'params/blockgroups_0/blocks_1/conv2/kernel',
 'params/blockgroups_0/blocks_1/norm2/scale',
 'params/blockgroups_0/blocks_1/norm2/bias',
 'params/blockgroups_0/blocks_2/conv1/kernel',
 'params/blockgroups_0/blocks_2/norm1/scale',
 'params/blockgroups_0/blocks_2/norm1/bias',
 'params/blockgroups_0/blocks_2/conv2/kernel',
 'params/blockgroups_0/blocks_2/norm2/scale',
 'params/blockgroups_0/blocks_2/norm2/bias',
 'params/blockgroups_1/blocks_0/conv1/kernel',
 'params/blockgroups_1/blocks_0/norm1/scale',
 'params/b

In [11]:
permutation_spec = resnet20_permutation_spec()

In [22]:
final_permutation = weight_matching(
    random.PRNGKey(0), permutation_spec,
    # flatten_params(model_a), 
    # flatten_params(model_b)
    flatten_params(a_sd['params']), 
    flatten_params(b_sd['params'])
)

2023-01-16 17:33:29.148591: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:219] failed to create cublas handle: cublas error
2023-01-16 17:33:29.148662: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:221] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2023-01-16 17:33:29.149941: E external/org_tensorflow/tensorflow/compiler/xla/status_macros.cc:57] INTERNAL: RET_CHECK failure (external/org_tensorflow/tensorflow/compiler/xla/service/gpu/gemm_algorithm_picker.cc:327) stream->parent()->GetBlasGemmAlgorithms(stream, &algorithms) 
*** Begin stack trace ***
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	
	_PyMethodDef_RawFastCallKeywords
	_PyObject_FastCallKeywords
	
	_PyEval_EvalFrameDefault
	_PyFunction_FastCa

XlaRuntimeError: INTERNAL: RET_CHECK failure (external/org_tensorflow/tensorflow/compiler/xla/service/gpu/gemm_algorithm_picker.cc:327) stream->parent()->GetBlasGemmAlgorithms(stream, &algorithms) 

In [26]:
flatten_params(a_sd['batch_stats']).keys()

dict_keys(['norm1/mean', 'norm1/var', 'blockgroups_0/blocks_0/norm1/mean', 'blockgroups_0/blocks_0/norm1/var', 'blockgroups_0/blocks_0/norm2/mean', 'blockgroups_0/blocks_0/norm2/var', 'blockgroups_0/blocks_1/norm1/mean', 'blockgroups_0/blocks_1/norm1/var', 'blockgroups_0/blocks_1/norm2/mean', 'blockgroups_0/blocks_1/norm2/var', 'blockgroups_0/blocks_2/norm1/mean', 'blockgroups_0/blocks_2/norm1/var', 'blockgroups_0/blocks_2/norm2/mean', 'blockgroups_0/blocks_2/norm2/var', 'blockgroups_1/blocks_0/norm1/mean', 'blockgroups_1/blocks_0/norm1/var', 'blockgroups_1/blocks_0/norm2/mean', 'blockgroups_1/blocks_0/norm2/var', 'blockgroups_1/blocks_0/shortcut/layers_1/mean', 'blockgroups_1/blocks_0/shortcut/layers_1/var', 'blockgroups_1/blocks_1/norm1/mean', 'blockgroups_1/blocks_1/norm1/var', 'blockgroups_1/blocks_1/norm2/mean', 'blockgroups_1/blocks_1/norm2/var', 'blockgroups_1/blocks_2/norm1/mean', 'blockgroups_1/blocks_2/norm1/var', 'blockgroups_1/blocks_2/norm2/mean', 'blockgroups_1/blocks_2/n

In [27]:
flatten_params(a_sd['params']).keys()

dict_keys(['conv1/kernel', 'norm1/scale', 'norm1/bias', 'blockgroups_0/blocks_0/conv1/kernel', 'blockgroups_0/blocks_0/norm1/scale', 'blockgroups_0/blocks_0/norm1/bias', 'blockgroups_0/blocks_0/conv2/kernel', 'blockgroups_0/blocks_0/norm2/scale', 'blockgroups_0/blocks_0/norm2/bias', 'blockgroups_0/blocks_1/conv1/kernel', 'blockgroups_0/blocks_1/norm1/scale', 'blockgroups_0/blocks_1/norm1/bias', 'blockgroups_0/blocks_1/conv2/kernel', 'blockgroups_0/blocks_1/norm2/scale', 'blockgroups_0/blocks_1/norm2/bias', 'blockgroups_0/blocks_2/conv1/kernel', 'blockgroups_0/blocks_2/norm1/scale', 'blockgroups_0/blocks_2/norm1/bias', 'blockgroups_0/blocks_2/conv2/kernel', 'blockgroups_0/blocks_2/norm2/scale', 'blockgroups_0/blocks_2/norm2/bias', 'blockgroups_1/blocks_0/conv1/kernel', 'blockgroups_1/blocks_0/norm1/scale', 'blockgroups_1/blocks_0/norm1/bias', 'blockgroups_1/blocks_0/conv2/kernel', 'blockgroups_1/blocks_0/norm2/scale', 'blockgroups_1/blocks_0/norm2/bias', 'blockgroups_1/blocks_0/shortcut

In [28]:
a_sd_union = a_sd['params']
a_sd_union.update(a_sd['batch_stats'])